### Лабораторная работа 4. Задача 2
#### Вариант 23. Метод с возвратом при неудачном шаге
##### Условие задания
Склад оптовой торговли отпускает 5 видов товаров. Известны потребности
Vi, издержки заказывания Ki, издержки содержания si, расход складской
площади на единицу товара fi, а также величина складской площади
торгового зала F. Хотя бы одна единица товара каждого вида должна
храниться на складе.
Требуется определить оптимальные партии поставок при ограничении на
максимальный уровень запаса при условии, что все пять видов продукции
поступают на склад от разных поставщиков (раздельная оптимизация).
1) Решить указанным в задании методом. Выводить промежуточные
результаты вычислений (координаты точки и значения функции в точке,
полученные на каждой итерации). Выписать полученный ответ.

##### Формула издержек (L)
$\displaystyle L(q) = (\frac{120*48000}{q_{1}} + \frac{200*q_{1}}{2})+(\frac{160*22400}{q_{2}} + \frac{280*q_{2}}{2})+(\frac{130*6400}{q_{3}} + \frac{260*q_{3}}{2})+(\frac{140*8600}{q_{4}} + \frac{200*q_{4}}{2})+(\frac{110*2460}{q_{5}} + \frac{250*q_{5}}{2})$

##### Алгоритм метода с возвратом при неудачном шаге
1. Задать начальную точку $x^{0}$, коэффициент сжатия $0 < \beta < 1$, $M$ - максимальное число неудачно выполненных испытаний на текущей итерации, $t_{0}$ - начальную величину шага, $R$ - минимальную величину шага, $N$ - максимальное число итераций. Положить $k = 0$, $j = 1$.
2. Получить случайный вектор $\xi^{j}=(\xi^{j}_{1},...,\xi^{j}_{n})^{T}$, где $\xi^{j}_{i}$ - случайная величина, равномерно распределённая на интервале $[-1,1]$.
3. Вычислить $y^{j} = x^{k} + t_{k}\frac{\xi^{j}}{\left \| \xi^{j} \right \|}$.
4. Проверить выполнение условий:
    * если $f(y^{j}) < f(x^{k})$, шаг удачный. Положить $x^{k+1}=y^{j}$, $t_{k+1}=t_{k}$, $k=k+1$ и проверить условие окончания. Если $k<N$, положить $j=1$ и перейти к шагу 2. Если $k=N$, поиск завершить: $x^{*} \cong x^{k}$;
    * если $f(y^{j}) \geq f(x^{k})$, шаг неудачный и перейти к шагу 5.
5. Оценить число неудачных шагов из текущей точки:
    * если $j < M$, следует положить $j=j+1$ и перейти к шагу 2;
    * если $j = M$, проверить условие окончания:
        * если $t_{k} \leq R$, процесс закончить: $x^{*} \cong x^{k}$, $f(x^{*}) \cong f(x^{k})$;
        * $t_{k} > R$, положить $t_{k} = \beta t_{k}$, $j = 1$ и перейти к шагу 2.

##### Начальные значения
* $N = 200$
* $\beta = 0.5$
* $M = 10$
* $t_{0} = 10$
* $R = 0.1$
* $x^{0} = [1, 1, 1, 1, 1]$

##### Алгоритм решения

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel("Задание2.xlsx", header=None, names=['v', 'k', 's'], usecols='B:D', skiprows=1, nrows=5)
df.head()

,v,k,s
0,48000,120,200
1,22400,160,280
2,6400,130,260
3,8600,140,200
4,2460,110,250


In [3]:
import task2_lib as lib

In [4]:
def generate_func(df):
    def L(q : np.ndarray):
        def f(v, k, s, q):
            return k * v / q + 0.5 * s * q
        return np.sum(np.vectorize(f)(df['v'], df['k'], df['s'], q))
    return L

L_func = generate_func(df)

In [ ]:
q = np.array([1, 1, 1, 1, 1], dtype=np.float64)
func_min_finder = lib.FuncMinFinder(L_func)
q = func_min_finder.return_on_an_unsuccessful_step(q, N=200)

##### Результат:

In [9]:
print("q =", lib.get_str_cords(q))
print("L =", L_func(q))

q = [x0 = 239.93888913795735,x1 = 159.95894034358054,x2 = 79.99146659440059,x3 = 109.70798380025687,x4 = 46.510918112891765]
L = 147177.24378656517


#### Приложение 1 (исходный код алгоритма метода из task2_lib.py)
```python
from random import uniform
import numpy as np
import pandas as pd

class Cords(np.ndarray):
    def __str__(self) -> str:
        cords_str_list = [f"x{i} = {self.__getitem__(i)}" for i in range(self.size)]
        return '[' + ','.join(cords_str_list) + ']'

def get_str_cords(cords):
    cords_str_list = [f"x{i} = {cords[i]}" for i in range(len(cords))]
    return '[' + ','.join(cords_str_list) + ']'

class FuncMinFinder:
    print_tries = False

    def __init__(self, func) -> None:
        self.func = func        

    def return_on_an_unsuccessful_step(self, x0 : np.ndarray, betta=0.5, M=10, t0=10, R=0.1, N=10) -> np.ndarray:
        cords_num = len(x0)
        
        k = 0
        j = 1 
        xk = x0
        tk = t0
        is_k_reached_N = False
        while not is_k_reached_N:
            print(f"Итерация {k}")
            print(f"\tx{k}: {get_str_cords(xk)}")   
            f_xk = self.func(xk)      
            print(f"\tf(x{k}): {f_xk}")   
            print(f"\tt{k}: {tk}\n")   
            while True:
                if self.print_tries:
                    print(f"\tПопытка {j}")
                y = 0                
                while True:
                    rand_vec = np.array([uniform(-1, 1) for _ in range(cords_num)])
                    vec_norm = np.linalg.norm(rand_vec, ord=2)
                    y : np.ndarray = xk + tk * (rand_vec / vec_norm)
                    if ((y > 0).all()):
                        break
                if self.print_tries:
                    print(f"\t\ty: {get_str_cords(y)}")
                f_y = self.func(y)
                if self.print_tries:
                    print(f"\t\tf(y): {f_y}")
                is_new_f_less = f_y < f_xk
                inequality_symb = '<' if is_new_f_less else '>='
                if self.print_tries:
                    print(f"\t\tf(y) {inequality_symb} f(x{k}) ({f_y} {inequality_symb} {f_xk})")
                if (is_new_f_less):
                    xk = y
                    k += 1
                    is_k_reached_N = k == N
                    j = 1                        
                    break
                if j < M:
                    j += 1
                    continue
                if self.print_tries:
                    print("\n\t\tj = M")
                if tk <= R:
                    if self.print_tries:
                        print(f"\t\t{k} <= R ({tk} <= {R})")
                        print("\t\tОстановка процесса оптимизации.")
                    is_k_reached_N = True
                    break
                tk *= betta
                if self.print_tries:
                    print (f"\t\tt{k} = t{k} * betta = {tk}")                
                    print(f"\t\tНачало новых попыток итерации {k}.\n")
                j = 1

        print(f"x_min = {get_str_cords(xk)}")
        print(f"f_min = {f_xk}")
        return xk
```

#### Приложение 2 (полный вывод результатов каждой итерации алгоритма)
***
Итерация 0
	x0: [x0 = 1.0,x1 = 1.0,x2 = 1.0,x3 = 1.0,x4 = 1.0]
	f(x0): 11651195.0
	t0: 10

Итерация 1
	x1: [x0 = 5.111143462690093,x1 = 5.065685284226297,x2 = 8.385083712107914,x3 = 3.738970245614306,x4 = 3.127741327700381]
	f(x1): 2345283.801943963
	t1: 10

Итерация 2
	x2: [x0 = 12.36404387416391,x1 = 3.549280552333948,x2 = 13.463810254476064,x3 = 2.005627030990853,x4 = 7.16482075700339]
	f(x2): 2180103.039246074
	t2: 10

Итерация 3
	x3: [x0 = 19.4869149763687,x1 = 8.609294594182604,x2 = 13.47249597571642,x3 = 6.46169752367297,x4 = 9.115288772676712]
	f(x3): 996338.8073670467
	t3: 10

Итерация 4
	x4: [x0 = 18.72961456455491,x1 = 15.132843454186936,x2 = 20.857215073382942,x3 = 5.351300034691135,x4 = 10.165406681496957]
	f(x4): 844381.0441380995
	t4: 10

Итерация 5
	x5: [x0 = 23.705250024447746,x1 = 19.621608973577725,x2 = 16.090385804150042,x3 = 6.889513902871991,x4 = 4.687699798511528]
	f(x5): 718315.9021219147
	t5: 10

Итерация 6
	x6: [x0 = 29.96160126378821,x1 = 16.83095576038035,x2 = 9.022544262173458,x3 = 8.254135321034664,x4 = 5.80740875010383]
	f(x6): 697939.1865702815
	t6: 10

Итерация 7
	x7: [x0 = 25.13919407236004,x1 = 19.38372112372684,x2 = 11.680492046631505,x3 = 12.749813385394342,x4 = 12.361184012995452]
	f(x7): 611141.6801673885
	t7: 10

Итерация 8
	x8: [x0 = 32.42543797820194,x1 = 22.177833865495092,x2 = 14.625539087033186,x3 = 18.219797738036405,x4 = 11.6473543116364]
	f(x8): 496969.2662141787
	t8: 10

Итерация 9
	x9: [x0 = 35.87696024122815,x1 = 26.863101497619162,x2 = 10.52082010797776,x3 = 23.789898433245206,x4 = 15.920587470665005]
	f(x9): 453739.09050718334
	t9: 10

Итерация 10
	x10: [x0 = 40.88934697640908,x1 = 29.08679230738685,x2 = 16.51436066850527,x3 = 27.244751204103522,x4 = 20.618740541877262]
	f(x10): 387391.569779589
	t10: 10

Итерация 11
	x11: [x0 = 44.34655140918833,x1 = 34.37827481280605,x2 = 22.4618848433207,x3 = 25.94637020217685,x4 = 25.41343692106917]
	f(x11): 346168.7909018772
	t11: 10

Итерация 12
	x12: [x0 = 50.5334211789553,x1 = 38.25751392544934,x2 = 19.092884036352604,x3 = 31.65636042549692,x4 = 27.062676615934553]
	f(x12): 318713.74019446824
	t12: 10

Итерация 13
	x13: [x0 = 55.89861156769488,x1 = 44.39418226575974,x2 = 18.718086784065175,x3 = 30.371172549253142,x4 = 21.426733667674497]
	f(x13): 300449.7857938705
	t13: 10

Итерация 14
	x14: [x0 = 59.568563714407624,x1 = 42.53094965683714,x2 = 19.318172146271525,x3 = 36.61576107447744,x4 = 28.03769632031357]
	f(x14): 288153.7469282058
	t14: 10

Итерация 15
	x15: [x0 = 68.57713622006099,x1 = 46.19229959645197,x2 = 21.514835221760386,x3 = 36.15163746120259,x4 = 27.405716388311607]
	f(x15): 266593.1466032714
	t15: 10

Итерация 16
	x16: [x0 = 72.43893351192621,x1 = 50.658935968083824,x2 = 25.940991792573342,x3 = 29.48964233748937,x4 = 28.48397432703891]
	f(x16): 256881.58612023667
	t16: 10

Итерация 17
	x17: [x0 = 76.46827701742396,x1 = 56.65973658508565,x2 = 29.779971198109937,x3 = 24.8043225596231,x4 = 25.157595128466586]
	f(x17): 250890.2292321818
	t17: 10

Итерация 18
	x18: [x0 = 71.05137612761939,x1 = 61.31243158844454,x2 = 30.966219604867813,x3 = 31.703608914433243,x4 = 25.205307470147396]
	f(x18): 241138.88398078937
	t18: 10

Итерация 19
	x19: [x0 = 67.32834914783818,x1 = 63.97588770182986,x2 = 32.34053987927295,x3 = 40.202382723066634,x4 = 27.425027987000295]
	f(x19): 234455.68768336897
	t19: 10

Итерация 20
	x20: [x0 = 70.23601069615742,x1 = 58.690569978873796,x2 = 33.84918476285675,x3 = 39.25447869441748,x4 = 35.19911246400074]
	f(x20): 233980.2238469392
	t20: 10

Итерация 21
	x21: [x0 = 65.92095407406792,x1 = 66.87839489614807,x2 = 36.011401035356954,x3 = 42.330926118582056,x4 = 34.75178539181458]
	f(x21): 229513.80440944733
	t21: 10

Итерация 22
	x22: [x0 = 72.88698830335657,x1 = 61.27302678288085,x2 = 35.666396593301386,x3 = 42.98844010988862,x4 = 39.16798995199338]
	f(x22): 225460.65503105219
	t22: 10

Итерация 23
	x23: [x0 = 76.84804326324517,x1 = 58.384013221051426,x2 = 38.70069740568063,x3 = 50.02429569723179,x4 = 43.32171105220631]
	f(x23): 219459.9959032323
	t23: 10

Итерация 24
	x24: [x0 = 75.99558320556633,x1 = 64.48765171165623,x2 = 42.26652710316681,x3 = 54.705528272426804,x4 = 48.55524037490323]
	f(x24): 212299.23192597379
	t24: 10

Итерация 25
	x25: [x0 = 78.80385977128873,x1 = 70.7311200219126,x2 = 41.182216223164325,x3 = 59.627867503599006,x4 = 53.820934308090955]
	f(x25): 205013.04613547702
	t25: 10

Итерация 26
	x26: [x0 = 74.88226767372502,x1 = 76.47588405327437,x2 = 43.475077073934024,x3 = 64.24526940234306,x4 = 58.82505165971057]
	f(x26): 203887.62499082572
	t26: 10

Итерация 27
	x27: [x0 = 79.36591368813623,x1 = 76.63607470941665,x2 = 40.90668057784025,x3 = 63.45713035935569,x4 = 67.34875001540755]
	f(x27): 201419.8484578466
	t27: 10

Итерация 28
	x28: [x0 = 85.86431348447479,x1 = 71.72442558973552,x2 = 35.64743910039509,x3 = 64.8718678084786,x4 = 69.34509668972348]
	f(x28): 201270.52647441765
	t28: 10

Итерация 29
	x29: [x0 = 87.23501613979079,x1 = 77.76487870825983,x2 = 37.48889011152475,x3 = 58.425746179287465,x4 = 73.43051887952264]
	f(x29): 198107.41601661933
	t29: 10

Итерация 30
	x30: [x0 = 84.18467749763715,x1 = 82.80981736032037,x2 = 43.98853824079356,x3 = 56.49578683130859,x4 = 77.82071924806749]
	f(x30): 196510.98012928435
	t30: 10

Итерация 31
	x31: [x0 = 90.854841569964,x1 = 78.2812989986087,x2 = 46.120874083478434,x3 = 60.63624166341675,x4 = 74.17226610554886]
	f(x31): 192101.1031757276
	t31: 10

Итерация 32
	x32: [x0 = 94.76489022505504,x1 = 84.34010756989358,x2 = 46.366341136886476,x3 = 63.70070678539153,x4 = 80.38122396119878]
	f(x32): 187217.4652465736
	t32: 10

Итерация 33
	x33: [x0 = 98.20491015167508,x1 = 90.15096846535381,x2 = 43.36528413654081,x3 = 57.75133936384078,x4 = 83.54331863437586]
	f(x33): 185978.46254288175
	t33: 10

Итерация 34
	x34: [x0 = 100.07151076447545,x1 = 95.4612378549397,x2 = 47.07221963231662,x3 = 63.977631461757035,x4 = 87.51934140894637]
	f(x34): 181517.60133222016
	t34: 10

Итерация 35
	x35: [x0 = 105.18176523753921,x1 = 100.74994494256597,x2 = 43.36357308442786,x3 = 58.909790851476515,x4 = 84.97418740703453]
	f(x35): 179917.88620585817
	t35: 10

Итерация 36
	x36: [x0 = 102.29619495306005,x1 = 107.34717797809594,x2 = 41.96090582083926,x3 = 60.07570439275863,x4 = 78.27917653548587]
	f(x36): 179525.9048555937
	t36: 10

Итерация 37
	x37: [x0 = 105.33988339404695,x1 = 111.76326242891004,x2 = 36.353843302087675,x3 = 64.82946786569383,x4 = 82.42607194671571]
	f(x37): 179181.88379436848
	t37: 10

Итерация 38
	x38: [x0 = 112.07048784593454,x1 = 111.53397933162042,x2 = 41.530196703329956,x3 = 66.97656895730098,x4 = 77.6051057218397]
	f(x38): 173645.89217148718
	t38: 10

Итерация 39
	x39: [x0 = 111.04324463125708,x1 = 105.11920789276716,x2 = 47.64769961071199,x3 = 65.23375827443168,x4 = 73.44166389241292]
	f(x39): 173287.85876291685
	t39: 10

Итерация 40
	x40: [x0 = 115.28843321292855,x1 = 100.58956016954775,x2 = 51.267988798000225,x3 = 63.54384059316946,x4 = 66.69641742986312]
	f(x40): 171792.43161174082
	t40: 10

Итерация 41
	x41: [x0 = 121.39421631509109,x1 = 99.97888516503033,x2 = 48.37967102891506,x3 = 67.98565466306015,x4 = 60.84198347075894]
	f(x41): 169480.43486924074
	t41: 10

Итерация 42
	x42: [x0 = 122.00459626724323,x1 = 107.84847314930502,x2 = 52.34417791071061,x3 = 66.80819678698768,x4 = 56.30399940225003]
	f(x42): 166988.54404421386
	t42: 10

Итерация 43
	x43: [x0 = 124.77639371099485,x1 = 106.88232477587076,x2 = 48.30283741997357,x3 = 72.0823682012327,x4 = 63.17672660347169]
	f(x43): 166731.6417371573
	t43: 10

Итерация 44
	x44: [x0 = 130.87602802726335,x1 = 111.53297240947123,x2 = 44.83075082642933,x3 = 74.22208739963803,x4 = 68.12972107498004]
	f(x44): 165365.8622685202
	t44: 10

Итерация 45
	x45: [x0 = 137.1173662859728,x1 = 111.86853230381783,x2 = 42.541493741440995,x3 = 69.20790476904091,x4 = 62.602482085798854]
	f(x45): 164971.98570195312
	t45: 10

Итерация 46
	x46: [x0 = 131.02587084005702,x1 = 111.92706670126401,x2 = 46.614526028051365,x3 = 73.6479138005259,x4 = 57.4462252313293]
	f(x46): 164266.52881363983
	t46: 10

Итерация 47
	x47: [x0 = 137.01149404328754,x1 = 106.59606119054206,x2 = 47.675680655429254,x3 = 77.76215957633656,x4 = 53.23913268818037]
	f(x47): 162933.1498411918
	t47: 10

Итерация 48
	x48: [x0 = 135.74918366948646,x1 = 106.1270431773149,x2 = 49.77168194026895,x3 = 85.61880049535327,x4 = 47.576397774108784]
	f(x48): 162080.35192512537
	t48: 10

Итерация 49
	x49: [x0 = 134.68250549107128,x1 = 113.49664791163295,x2 = 47.36860606495261,x3 = 83.32873812259363,x4 = 53.367076869858536]
	f(x49): 161948.4452500055
	t49: 10

Итерация 50
	x50: [x0 = 136.9152935492052,x1 = 115.23398522494853,x2 = 46.870509122589795,x3 = 91.86473855734192,x4 = 49.02139941275224]
	f(x50): 160780.6431066656
	t50: 10

Итерация 51
	x51: [x0 = 140.86931897609284,x1 = 115.93391060612636,x2 = 55.30902126538023,x3 = 88.89238388031059,x4 = 50.97905307829609]
	f(x51): 158467.88110966678
	t51: 10

Итерация 52
	x52: [x0 = 147.56558909003488,x1 = 115.42910703855433,x2 = 59.449133188150526,x3 = 93.10050820600465,x4 = 46.50062669777686]
	f(x52): 156597.19605077174
	t52: 10

Итерация 53
	x53: [x0 = 150.495621915404,x1 = 118.845920107818,x2 = 63.955668512188545,x3 = 89.01748534591724,x4 = 53.03976729797575]
	f(x53): 155600.4556435212
	t53: 10

Итерация 54
	x54: [x0 = 156.11857297217435,x1 = 118.90915524457007,x2 = 70.16671575997458,x3 = 91.23902145781143,x4 = 48.05317606548388]
	f(x54): 154231.89492974954
	t54: 10

Итерация 55
	x55: [x0 = 162.34052121759598,x1 = 115.3789709256245,x2 = 71.4193424675948,x3 = 84.40499660507366,x4 = 48.79627050704871]
	f(x55): 154215.0534045488
	t55: 10

Итерация 56
	x56: [x0 = 163.0992206413886,x1 = 123.11429512459924,x2 = 73.39241748797541,x3 = 83.09428159847913,x4 = 42.9671937485448]
	f(x56): 153318.06556410802
	t56: 10

Итерация 57
	x57: [x0 = 167.87959866908147,x1 = 116.4598381165949,x2 = 77.46658885656558,x3 = 87.1252587504782,x4 = 43.1033506447884]
	f(x57): 153185.53218879257
	t57: 10

Итерация 58
	x58: [x0 = 170.68625930317,x1 = 113.02607083717753,x2 = 73.44811514979045,x3 = 93.71843363556845,x4 = 47.65459359132866]
	f(x58): 153077.90056107432
	t58: 10

Итерация 59
	x59: [x0 = 176.41579208621584,x1 = 116.39271090729163,x2 = 69.02003839270779,x3 = 99.30247435767654,x4 = 49.90155023833537]
	f(x59): 152121.2689222734
	t59: 10

Итерация 60
	x60: [x0 = 174.43092122662222,x1 = 124.91485926915892,x2 = 68.43781484418452,x3 = 97.31640135652442,x4 = 54.27759922602945]
	f(x60): 151572.15815620183
	t60: 10

Итерация 61
	x61: [x0 = 175.49514620829143,x1 = 128.9212372375568,x2 = 75.36633243325147,x3 = 95.07629627783642,x4 = 48.81921722976414]
	f(x61): 150873.30793381197
	t61: 10

Итерация 62
	x62: [x0 = 181.74592656776818,x1 = 134.641668337922,x2 = 73.28184839604371,x3 = 91.06735985394019,x4 = 51.60987449487902]
	f(x62): 150238.0251502099
	t62: 10

Итерация 63
	x63: [x0 = 180.95991682480863,x1 = 141.20228136540848,x2 = 69.95429600825499,x3 = 91.42122192186298,x4 = 44.89104110425574]
	f(x63): 150015.3685813055
	t63: 10

Итерация 64
	x64: [x0 = 183.9219348585702,x1 = 146.6955418071626,x2 = 66.41511054243009,x3 = 93.38226849948057,x4 = 38.206805811745575]
	f(x64): 149929.8172757695
	t64: 10

Итерация 65
	x65: [x0 = 189.91375584951518,x1 = 152.16854615891208,x2 = 60.71203615149494,x3 = 93.95557114835279,x4 = 39.3428388845292]
	f(x65): 149779.93936112884
	t65: 10

Итерация 66
	x66: [x0 = 188.371785383234,x1 = 156.4863180642142,x2 = 64.99544021640749,x3 = 96.92961658325504,x4 = 46.53913489169344]
	f(x66): 149222.55571205975
	t66: 10

Итерация 67
	x67: [x0 = 191.05913930524923,x1 = 151.11491498498762,x2 = 71.01544688273759,x3 = 99.15983716772186,x4 = 41.77344567183591]
	f(x67): 148832.0310139478
	t67: 10

Итерация 68
	x68: [x0 = 195.91631051961465,x1 = 151.44467386854055,x2 = 65.27011310586954,x3 = 101.7118617843657,x4 = 47.83788769245751]
	f(x68): 148736.63255888247
	t68: 10

Итерация 69
	x69: [x0 = 192.0205999076997,x1 = 157.72376974886828,x2 = 71.64494524927284,x3 = 101.63525800465854,x4 = 50.01780529599934]
	f(x69): 148602.21314890034
	t69: 10

Итерация 70
	x70: [x0 = 197.82721236095094,x1 = 154.28206473985986,x2 = 68.06354383569526,x3 = 106.3316025543981,x4 = 54.44000273999367]
	f(x70): 148532.67781291637
	t70: 10

Итерация 71
	x71: [x0 = 200.30720970027951,x1 = 158.394372741771,x2 = 76.48298568533681,x3 = 108.6943768844622,x4 = 55.12471618373147]
	f(x71): 148155.6798101138
	t71: 10

Итерация 72
	x72: [x0 = 205.6317367808998,x1 = 159.58873323214095,x2 = 73.14725166064684,x3 = 113.65836599407794,x4 = 49.25491167055332]
	f(x72): 147867.73957557115
	t72: 10

Итерация 73
	x73: [x0 = 207.29308133418402,x1 = 153.77155664844688,x2 = 76.72906695897622,x3 = 119.47681586237296,x4 = 45.16631396862844]
	f(x73): 147831.42984723664
	t73: 10

Итерация 74
	x74: [x0 = 208.66814051699222,x1 = 157.5975542818628,x2 = 84.55293346343805,x3 = 114.7666692854998,x4 = 45.435732783117615]
	f(x74): 147710.10054164444
	t74: 10

Итерация 75
	x75: [x0 = 209.52615407261234,x1 = 162.4073186314563,x2 = 78.08376793254641,x3 = 112.85559167361458,x4 = 50.969958956217724]
	f(x75): 147688.64052163446
	t75: 10

Итерация 76
	x76: [x0 = 214.40518976938085,x1 = 167.77169920175766,x2 = 80.55467851626683,x3 = 108.51139905310632,x4 = 46.23276325308156]
	f(x76): 147535.27402892674
	t76: 10

Итерация 77
	x77: [x0 = 221.5123315110886,x1 = 168.54975987075434,x2 = 79.43043790126029,x3 = 111.48559129207507,x4 = 40.00593248423426]
	f(x77): 147528.44710912651
	t77: 10

Итерация 78
	x78: [x0 = 225.27286937231705,x1 = 162.85363252825866,x2 = 77.32485853356842,x3 = 116.50924892904195,x4 = 44.878497287829504]
	f(x78): 147339.60422206248
	t78: 10

Итерация 79
	x79: [x0 = 231.54186989667605,x1 = 166.55871495546748,x2 = 79.45170092375281,x3 = 109.99635808108451,x4 = 45.05397059942442]
	f(x79): 147250.87556329995
	t79: 10

Итерация 80
	x80: [x0 = 233.40250679181264,x1 = 164.1856558672348,x2 = 76.95490773029924,x3 = 112.69535425722648,x4 = 46.59930690538104]
	f(x80): 147234.32432524202
	t80: 5.0

Итерация 81
	x81: [x0 = 231.8655550538187,x1 = 163.35480247076677,x2 = 80.16700266112991,x3 = 109.86282196980945,x4 = 44.70018369065093]
	f(x81): 147224.82151280495
	t81: 5.0

Итерация 82
	x82: [x0 = 236.21858078361313,x1 = 162.77056453753568,x2 = 78.92310765584226,x3 = 109.90235030078858,x4 = 46.740035587233585]
	f(x82): 147191.95425146728
	t82: 5.0

Итерация 83
	x83: [x0 = 237.91518443624733,x1 = 158.3219536801171,x2 = 80.09698616029111,x3 = 108.93533835875506,x4 = 46.87529328898615]
	f(x83): 147182.46965952622
	t83: 5.0

Итерация 84
	x84: [x0 = 238.59781732186588,x1 = 159.0766995901482,x2 = 80.59324378627612,x3 = 109.27432703204602,x4 = 46.46824783354464]
	f(x84): 147179.57843220545
	t84: 1.25

Итерация 85
	x85: [x0 = 238.5826177634002,x1 = 159.33572447837676,x2 = 80.26266980017193,x3 = 108.99567635029531,x4 = 46.837537310746036]
	f(x85): 147179.32837162958
	t85: 0.625

Итерация 86
	x86: [x0 = 238.6992725166494,x1 = 159.63994174029804,x2 = 80.61082054787047,x3 = 109.30880220879867,x4 = 46.58217133864526]
	f(x86): 147178.83175482767
	t86: 0.625

Итерация 87
	x87: [x0 = 239.06987180871175,x1 = 159.71420566855633,x2 = 80.267299802357,x3 = 109.21729476321178,x4 = 46.23376651880697]
	f(x87): 147178.2597099474
	t87: 0.625

Итерация 88
	x88: [x0 = 239.15009164837258,x1 = 160.04064827181566,x2 = 79.98627020236049,x3 = 109.55982707660743,x4 = 46.518931352381685]
	f(x88): 147177.56905723782
	t88: 0.625

Итерация 89
	x89: [x0 = 239.2733422379065,x1 = 159.76456597667263,x2 = 79.97705522554465,x3 = 109.51555867708377,x4 = 46.5837349131943]
	f(x89): 147177.55899671634
	t89: 0.3125

Итерация 90
	x90: [x0 = 239.42660909421517,x1 = 159.79598263663868,x2 = 79.73870437626402,x3 = 109.58808508389508,x4 = 46.478343702953225]
	f(x90): 147177.54874150202
	t90: 0.3125

Итерация 91
	x91: [x0 = 239.3106217374224,x1 = 160.06502860626614,x2 = 79.80882182815276,x3 = 109.5359554267506,x4 = 46.54302361543054]
	f(x91): 147177.5353512168
	t91: 0.3125

Итерация 92
	x92: [x0 = 239.4993683709802,x1 = 159.9670214281702,x2 = 79.98489230915474,x3 = 109.65701100313616,x4 = 46.62530535403612]
	f(x92): 147177.37569975026
	t92: 0.3125

Итерация 93
	x93: [x0 = 239.634023376285,x1 = 160.15846591068834,x2 = 80.05910249192098,x3 = 109.59203017090584,x4 = 46.44325143379219]
	f(x93): 147177.35876319435
	t93: 0.3125

Итерация 94
	x94: [x0 = 239.77434964513216,x1 = 159.99652220227503,x2 = 79.93204513737047,x3 = 109.76621028921065,x4 = 46.37074806408619]
	f(x94): 147177.33590072326
	t94: 0.3125

Итерация 95
	x95: [x0 = 239.9070566864112,x1 = 159.87919563023294,x2 = 80.04684088633711,x3 = 109.59355338778414,x4 = 46.523362274095625]
	f(x95): 147177.27579804807
	t95: 0.3125

Итерация 96
	x96: [x0 = 239.98464015566296,x1 = 159.8062514386309,x2 = 80.00108243784693,x3 = 109.6937657063337,x4 = 46.553983644298135]
	f(x96): 147177.27546285087
	t96: 0.15625

Итерация 97
	x97: [x0 = 239.93367119219118,x1 = 159.8959605836405,x2 = 79.9282288352065,x3 = 109.77170362674488,x4 = 46.50513152347873]
	f(x97): 147177.26242699876
	t97: 0.15625

Итерация 98
	x98: [x0 = 240.0138798388038,x1 = 160.00791655088014,x2 = 79.99309713586717,x3 = 109.79528435849225,x4 = 46.53125717309098]
	f(x98): 147177.2440851102
	t98: 0.15625

Итерация 99
	x99: [x0 = 240.0541064096177,x1 = 159.96513965346094,x2 = 80.00008670856245,x3 = 109.74553115839106,x4 = 46.54271320320478]
	f(x99): 147177.24280493316
	t99: 0.078125

x_min = [x0 = 240.0541064096177,x1 = 159.96513965346094,x2 = 80.00008670856245,x3 = 109.74553115839106,x4 = 46.54271320320478]
f_min = 147177.24280493316
***